In [2]:
import pygame
import numpy as np
import random
from sys import exit
from Agent.TriviaLC import triviaAgent
from Agent.HangmanLC import HangMan

pygame 2.5.2 (SDL 2.28.3, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


Ashkan

In [ ]:
# Initialize agents
trivia_agent = triviaAgent()
hangman_agent = HangMan()

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((800, 800))
pygame.display.set_caption("Maropelleh (Snakes, Ladders, Trivia, Hangman)")
clock = pygame.time.Clock()

# Load assets
sky_background = pygame.image.load('graphics/blue-sky.png')

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
GRAY = (200, 200, 200)
YELLOW = (255, 255, 0)
LIGHT_YELLOW = (255, 255, 200)
ORANGE = (255, 200, 0)

# Game constants
BOARD_SIZE = 10
CELL_SIZE = 60
GRID_OFFSET = 100
PLAYER_SIZE = 15
BUTTON_WIDTH = 150
BUTTON_HEIGHT = 50

# Game state
player1_pos = 0
player2_pos = 0
current_player = 1
game_over = False
winner = None
message = "Player 1's turn. Press the button to roll!"
message_timer = 0
message_duration = 120

# Mini-game modal
active_modal = None  # 'trivia', 'hangman', or None
modal_data = None
user_answer = ""

# Snakes and ladders
snakes = {}
ladders = {}
for _ in range(5):
    start = random.randint(20, 98)
    end = random.randint(1, start - 1)
    snakes[end] = start
for _ in range(5):
    start = random.randint(1, 80)
    end = random.randint(start + 1, 99)
    ladders[start] = end

# Mini-game tiles
total_cells = list(range(1, 100))
random.shuffle(total_cells)
trivia_cells = set(total_cells[:15])
hangman_cells = set(total_cells[15:30])

# Fonts
font = pygame.font.SysFont(None, 36)
small_font = pygame.font.SysFont(None, 24)


def get_cell_position(pos):
    row = (pos - 1) // BOARD_SIZE
    col = (pos - 1) % BOARD_SIZE
    x = col * CELL_SIZE + GRID_OFFSET + CELL_SIZE // 2
    y = (BOARD_SIZE - 1 - row) * CELL_SIZE + GRID_OFFSET + CELL_SIZE // 2
    return x, y


def draw_board():
    screen.fill(WHITE)
    screen.blit(sky_background, (0, 0))
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            cell_num = row * BOARD_SIZE + col + 1
            x = col * CELL_SIZE + GRID_OFFSET
            y = (BOARD_SIZE - 1 - row) * CELL_SIZE + GRID_OFFSET
            color = BLUE if cell_num in trivia_cells else YELLOW if cell_num in hangman_cells else LIGHT_YELLOW if (row + col) % 2 == 0 else ORANGE
            pygame.draw.rect(screen, color, (x, y, CELL_SIZE, CELL_SIZE))
            pygame.draw.rect(screen, BLACK, (x, y, CELL_SIZE, CELL_SIZE), 1)
            text = font.render(str(cell_num), True, BLACK)
            screen.blit(text, text.get_rect(center=(x + CELL_SIZE//2, y + CELL_SIZE//2)))
            if cell_num in trivia_cells:
                screen.blit(small_font.render("T", True, WHITE), (x + CELL_SIZE//2 - 5, y + CELL_SIZE//2 + 15))
            elif cell_num in hangman_cells:
                screen.blit(small_font.render("H", True, BLACK), (x + CELL_SIZE//2 - 5, y + CELL_SIZE//2 + 15))


def draw_ladder(start, end):
    x1, y1 = get_cell_position(start)
    x2, y2 = get_cell_position(end)
    steps = 5
    dx, dy = (x2 - x1) / steps, (y2 - y1) / steps
    offset = 5
    pygame.draw.line(screen, GREEN, (x1 - offset, y1), (x2 - offset, y2), 3)
    pygame.draw.line(screen, GREEN, (x1 + offset, y1), (x2 + offset, y2), 3)
    for i in range(steps + 1):
        rx = x1 + dx * i
        ry = y1 + dy * i
        pygame.draw.line(screen, GREEN, (rx - offset, ry), (rx + offset, ry), 2)


def draw_snake(start, end):
    x1, y1 = get_cell_position(start)
    x2, y2 = get_cell_position(end)
    mid_x = (x1 + x2) // 2
    mid_y = (y1 + y2) // 2
    points = [(x1, y1), (mid_x + 20, mid_y - 30), (mid_x - 20, mid_y + 30), (x2, y2)]
    pygame.draw.lines(screen, RED, False, points, 4)
    pygame.draw.polygon(screen, RED, [(x1, y1), (x1 - 5, y1 - 10), (x1 + 5, y1 - 10)])


def draw_snakes_and_ladders():
    for start, end in ladders.items():
        draw_ladder(start, end)
    for end, start in snakes.items():
        draw_snake(start, end)


def draw_players():
    pygame.draw.circle(screen, RED, (get_cell_position(player1_pos + 1)[0] - 10, get_cell_position(player1_pos + 1)[1]), PLAYER_SIZE)
    pygame.draw.circle(screen, BLUE, (get_cell_position(player2_pos + 1)[0] + 10, get_cell_position(player2_pos + 1)[1]), PLAYER_SIZE)


def draw_button():
    rect = pygame.Rect(330, 720, BUTTON_WIDTH, BUTTON_HEIGHT)
    pygame.draw.rect(screen, YELLOW, rect)
    pygame.draw.rect(screen, BLACK, rect, 2)
    screen.blit(font.render("Roll Dice", True, BLACK), font.render("Roll Dice", True, BLACK).get_rect(center=rect.center))
    return rect


def draw_game_info():
    screen.blit(font.render(f"Current Player: {current_player}", True, BLACK), (20, 20))
    if game_over:
        screen.blit(font.render(f"Player {winner} wins!", True, BLACK), (20, 50))
    elif message_timer > 0:
        screen.blit(font.render(message, True, BLACK), (20, 70))


def draw_modal():
    if active_modal and modal_data:
        rect = pygame.Rect(100, 200, 600, 400)
        pygame.draw.rect(screen, GRAY, rect)
        pygame.draw.rect(screen, BLACK, rect, 3)
        lines = modal_data.split("\n")
        for i, line in enumerate(lines):
            screen.blit(small_font.render(line.strip(), True, BLACK), (rect.x + 20, rect.y + 30 + 25 * i))
        if active_modal == "trivia":
            screen.blit(small_font.render(f"Your answer: {user_answer}", True, BLUE), (rect.x + 20, rect.y + 330))
        screen.blit(small_font.render("Press ENTER to continue...", True, BLACK), (rect.x + 20, rect.y + 360))


def roll_dice():
    return random.randint(1, 6)


def move_player(pos, steps):
    new_pos = pos + steps
    if new_pos in ladders: new_pos = ladders[new_pos]
    if new_pos in snakes: new_pos = snakes[new_pos]
    return min(new_pos, 99), f"Player {current_player} moved to {new_pos+1}"


def trigger_mini_game(cell):
    global active_modal, modal_data
    if cell in trivia_cells:
        active_modal = "trivia"
        topics = ['Sports', 'literature', 'Movies', 'Celebrities', 'Music', 'general knowledge']
        modal_data = trivia_agent.envoke(topics[random.randint(0,len(topics))])
    elif cell in hangman_cells:
        active_modal = "hangman"
        modal_data = hangman_agent.envoke()

# Main Loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit(); exit()
        if active_modal == "trivia":
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    active_modal = None
                elif event.key == pygame.K_BACKSPACE:
                    user_answer = user_answer[:-1]
                    trivia_agent.check_answer(user_answer)
                elif event.unicode.isprintable():
                    user_answer += event.unicode
        elif active_modal:
            if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                active_modal = None
        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over:
            if draw_button().collidepoint(pygame.mouse.get_pos()):
                dice = roll_dice()
                if current_player == 1:
                    player1_pos, message = move_player(player1_pos, dice)
                    if player1_pos == 99: game_over = True; winner = 1
                    trigger_mini_game(player1_pos + 1)
                else:
                    player2_pos, message = move_player(player2_pos, dice)
                    if player2_pos == 99: game_over = True; winner = 2
                    trigger_mini_game(player2_pos + 1)
                message_timer = message_duration
                current_player = 3 - current_player
    if message_timer > 0:
        message_timer -= 1

    draw_board()
    draw_snakes_and_ladders()
    draw_players()
    draw_game_info()
    draw_button()
    draw_modal()
    pygame.display.update()
    clock.tick(60)

2025-05-04 22:43:24.806 Python[24851:8688375] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/pd/wynthhk510v_lw3l_4q234gh0000gn/T/org.python.python.savedState
2025-05-04 22:43:25.246 Python[24851:8688375] +[IMKClient subclass]: chose IMKClient_Legacy
2025-05-04 22:43:25.246 Python[24851:8688375] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


SystemExit: 

/Users/rav007/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

Brandon

In [ ]:
import pygame
import requests
import sys
import os
import time
import atexit
import signal
import Front_Game_Client.game_client
pygame.init()

SCREEN_WIDTH, SCREEN_HEIGHT = 900, 750
FONT = pygame.font.SysFont("Arial", 22)
# SERVER = "http://127.0.0.1:5000"
SERVER="http://10.0.0.120:5001"
WHITE = (255, 255, 255)
GRAY = (240, 240, 240)
BLUE = (80, 120, 255)
BLACK = (0, 0, 0)
BUTTON_COLOR = (30, 144, 255)
RED = (200, 50, 50)
GREEN = (50, 200, 100)
ORANGE = (255, 140, 0)

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Staircase Game Menu")

user_id = ""
user_info = {}
friends = []
friend_requests = []
search_result = None
search_text = ""
message = ""
party_members = []
pending_invite_from = None

# Load logo
logo = None
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))
    logo_path = os.path.join(base_dir, "../BackEnd/logo.png")
    if os.path.exists(logo_path):
        logo = pygame.image.load(logo_path)
        logo = pygame.transform.scale(logo, (150, 150))
except:
    pass

def update_status(is_online=True):
    try:
        requests.post(f"{SERVER}/set_status", json={
            "username": user_id,
            "status": is_online
        })
    except:
        pass

atexit.register(lambda: update_status(False))
signal.signal(signal.SIGINT, lambda sig, frame: sys.exit(0))
signal.signal(signal.SIGTERM, lambda sig, frame: sys.exit(0))

def start_game():
    try:
        res = requests.post(f"{SERVER}/join")
        if res.status_code == 200:
            player_id = res.json()["player_id"]
            print("Joined game as:", player_id)
            # Now launch the game loop or load game scene
            # You can import and call a game function here
            import game_client
            game_client.main(player_id)
        else:
            print("Failed to join game.")
    except Exception as e:
        print("Error starting game:", str(e))

def send_game_invite(friend_username):
    try:
        res = requests.post(f"{SERVER}/send_invite", json={
            "from": user_info["Username"],
            "to": friend_username
        })
        return res.status_code == 200
    except:
        return False

def fetch_party():
    global party_members
    try:
        res = requests.get(f"{SERVER}/get_party", params={"username": user_info["Username"]})
        if res.status_code == 200:
            party_members = res.json().get("party", [])
    except:
        party_members = []

def check_for_invite():
    global pending_invite_from
    try:
        res = requests.get(f"{SERVER}/check_invite", params={"username": user_info["Username"]})
        if res.status_code == 200:
            from_user = res.json().get("from")
            if from_user:
                pending_invite_from = from_user
    except:
        pass

def draw_text(text, x, y, color=BLACK):
    rendered = FONT.render(text, True, color)
    screen.blit(rendered, (x, y))

def draw_button(text, x, y, w, h, color=BUTTON_COLOR):
    pygame.draw.rect(screen, color, (x, y, w, h), border_radius=8)
    txt = FONT.render(text, True, WHITE)
    screen.blit(txt, (x + 10, y + 10))
    return pygame.Rect(x, y, w, h)

def fetch_user_info():
    global user_info
    try:
        res = requests.get(f"{SERVER}/get_use_info", params={"user_id": user_id})
        user_info.update(res.json())
        if "Username" not in user_info:
            user_info["Username"] = user_id
    except:
        user_info = {"Points": 0, "NumberOfWins": 0, "Username": user_id}

def fetch_friend_data():
    global friends, friend_requests
    try:
        res = requests.get(f"{SERVER}/get_friends_list", params={"user_id": user_info["Username"]})
        friends = res.json().get("friends", [])

        req = requests.get(f"{SERVER}/get_friend_requests", params={"user_id": user_info["Username"]})
        received_ids = req.json().get("received", [])

        friend_requests = []
        for uid in received_ids:
            user_doc = requests.get(f"{SERVER}/get_username_by_id", params={"doc_id": uid})
            if user_doc.status_code == 200:
                data = user_doc.json()
                friend_requests.append({
                    "id": uid,
                    "Username": data.get("Username", uid)
                })
    except:
        friends = []
        friend_requests = []

def search_user_by_username(username):
    try:
        res = requests.get(f"{SERVER}/search_user_by_username", params={"username": username})
        if res.status_code == 200:
            return res.json()["user"]
    except:
        pass
    return None

def send_friend_request(receiver_username):
    try:
        res = requests.post(f"{SERVER}/send_friend_request", json={
            "sender_username": user_info.get("Username"),
            "receiver_username": receiver_username
        })
        return res.status_code == 200
    except:
        return False

def accept_friend(sender_id):
    try:
        me = requests.get(f"{SERVER}/search_user_by_username", params={"username": user_info["Username"]})
        if me.status_code == 200:
            user_doc = me.json()
            user_doc_id = user_doc["user"]["id"]
            requests.post(f"{SERVER}/accept_friend_request", json={
                "user_id": user_doc_id,
                "sender_id": sender_id
            })
    except:
        pass

def reject_friend(sender_id):
    try:
        me = requests.get(f"{SERVER}/search_user_by_username", params={"username": user_info["Username"]})
        if me.status_code == 200:
            user_doc = me.json()
            user_doc_id = user_doc["user"]["id"]
            requests.post(f"{SERVER}/reject_friend_request", json={
                "user_id": user_doc_id,
                "sender_id": sender_id
            })
    except:
        pass

def login_screen():
    global user_id
    input_box_id = pygame.Rect(320, 230, 240, 40)
    input_box_pw = pygame.Rect(320, 300, 240, 40)
    active_box = None
    id_text, pw_text = '', ''
    show_error = False

    while True:
        screen.fill(WHITE)
        if logo:
            screen.blit(logo, (SCREEN_WIDTH // 2 - logo.get_width() // 2, 40))
        draw_text("User ID (Username):", 320, 200)
        draw_text("Password:", 320, 270)
        pygame.draw.rect(screen, GRAY if active_box != 'id' else BLUE, input_box_id, 2)
        screen.blit(FONT.render(id_text, True, BLACK), (input_box_id.x+5, input_box_id.y+5))
        pygame.draw.rect(screen, GRAY if active_box != 'pw' else BLUE, input_box_pw, 2)
        screen.blit(FONT.render('*' * len(pw_text), True, BLACK), (input_box_pw.x+5, input_box_pw.y+5))
        draw_text("Press Enter when ready", 340, 360, ORANGE)

        if show_error:
            draw_text("Missing credentials!", 340, 390, RED)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                update_status(False)
                pygame.quit(); sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                active_box = 'id' if input_box_id.collidepoint(event.pos) else 'pw' if input_box_pw.collidepoint(event.pos) else None
            if event.type == pygame.KEYDOWN and active_box:
                if event.key == pygame.K_RETURN:
                    if id_text and pw_text:
                        user_id = id_text.strip()
                        fetch_user_info()
                        update_status(True)
                        fetch_friend_data()
                        fetch_party()
                        return
                    else:
                        show_error = True
                elif event.key == pygame.K_BACKSPACE:
                    if active_box == 'id': id_text = id_text[:-1]
                    elif active_box == 'pw': pw_text = pw_text[:-1]
                else:
                    if active_box == 'id': id_text += event.unicode
                    elif active_box == 'pw': pw_text += event.unicode
        pygame.display.update()
        pygame.time.delay(50)
def menu_screen():
    global search_text, search_result, message, pending_invite_from
    clock = pygame.time.Clock()
    input_box = pygame.Rect(500, 440, 220, 35)
    active = False
    last_refresh = 0
    last_check = 0

    while True:
        screen.fill((250, 250, 255))
        draw_text(f"User: {user_info.get('Username', user_id)}", 20, 20)
        draw_text(f"Wins: {user_info.get('NumberOfWins', 0)}", 20, 60)
        draw_text(f"Points: {user_info.get('Points', 0)}", 20, 100)

        if time.time() - last_refresh > 5:
            fetch_friend_data()
            fetch_party()
            check_for_invite()
            last_refresh = time.time()

        #  Auto-start check
        # if time.time() - last_check > 3:
        #     try:
        #         res = requests.get(f"{SERVER}/get_party_status", params={"username": user_info["Username"]})
        #         if res.status_code == 200 and res.json().get("game_started"):
        #             print("Game started remotely. Launching...")
        #             import importlib.util
        #             import sys
        #             game_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "game_client.py"))
        #             spec = importlib.util.spec_from_file_location("game_client", game_path)
        #             game_module = importlib.util.module_from_spec(spec)
        #             sys.modules["game_client"] = game_module
        #             spec.loader.exec_module(game_module)
        #             game_module.main()
        #     except Exception as e:
        #         print("Auto-start error:", e)
        #     last_check = time.time()

        # Friends List
        draw_text("Friends:", 40, 140)
        for i, f in enumerate(friends[:5]):
            y = 170 + i * 50
            pygame.draw.rect(screen, (220, 235, 255), (40, y, 360, 40), border_radius=10)
            draw_text(f['Username'], 80, y + 10)
            dot_color = GREEN if f.get("Online") else RED
            pygame.draw.circle(screen, dot_color, (55, y + 20), 8)
            if f.get("Online"):
                invite_btn = draw_button("Invite", 260, y + 5, 80, 28, ORANGE)
                if pygame.mouse.get_pressed()[0]:
                    if invite_btn.collidepoint(pygame.mouse.get_pos()):
                        if send_game_invite(f["Username"]):
                            message = f"Invited {f['Username']}!"
                        else:
                            message = f"Failed to invite {f['Username']}"

        # Party List
        draw_text("Party:", 40, 460)
        reset_btn = draw_button("Reset", 300, 460, 80, 30, RED)
        pygame.draw.rect(screen, (245, 255, 245), pygame.Rect(40, 490, 360, 100), border_radius=12)
        for i, member in enumerate(party_members):
            draw_text(f"- {member}", 60, 500 + i * 20, BLACK)

        # Friend Requests
        draw_text("Requests:", 500, 140)
        for i, sender in enumerate(friend_requests[:5]):
            y = 170 + i * 60
            pygame.draw.rect(screen, (255, 240, 240), (500, y, 360, 50), border_radius=10)
            draw_text(sender["Username"], 520, y + 15)

            acc_btn = draw_button("Accept", 700, y + 10, 70, 30, GREEN)
            rej_btn = draw_button("Reject", 770, y + 10, 70, 30, RED)

            if pygame.mouse.get_pressed()[0]:
                mouse_pos = pygame.mouse.get_pos()
                if acc_btn.collidepoint(mouse_pos):
                    accept_friend(sender["id"])
                    fetch_friend_data()
                elif rej_btn.collidepoint(mouse_pos):
                    reject_friend(sender["id"])
                    fetch_friend_data()

        # Search Box
        pygame.draw.rect(screen, GRAY if not active else BLUE, input_box, 2)
        screen.blit(FONT.render(search_text, True, BLACK), (input_box.x + 5, input_box.y + 5))
        draw_text("Search by Username:", 500, 410)
        search_btn = draw_button("Search", 730, 440, 100, 35)

        if search_result == "not_found":
            draw_text("User not found.", 500, 500, RED)
        elif search_result == "already_friends":
            draw_text("Already your friend or request sent.", 500, 500, ORANGE)
        elif search_result and isinstance(search_result, dict):
            draw_text(f"Found: {search_result['Username']}", 500, 500)
            req_btn = draw_button("Send Request", 650, 495, 150, 35)
            if pygame.mouse.get_pressed()[0]:
                if req_btn.collidepoint(pygame.mouse.get_pos()):
                    if send_friend_request(search_result["Username"]):
                        message = "Friend request sent."
                        search_result = None
                        fetch_friend_data()
                    else:
                        message = "Failed to send request."
        if message:
            draw_text(message, 500, 530, GREEN)

        # Start Game Button
        start_btn = draw_button("Start Game", 350, 610, 200, 45)
        if pygame.mouse.get_pressed()[0] and start_btn.collidepoint(pygame.mouse.get_pos()):
            try:
                party_res = requests.get(f"{SERVER}/get_party", params={"username": user_info["Username"]})
                party = party_res.json().get("party", [])

                if user_info["Username"] not in party:
                    party.append(user_info["Username"])
                # Restart backend state BEFORE players join
                # Reset Firestore party list
                requests.post(f"{SERVER}/reset_party", json={"username": user_info["Username"]})

                # Restart backend game state
                restart_res = requests.post(f"{SERVER}/restart")
                if restart_res.status_code == 200:
                    print("✅ Game backend restarted.")
                else:
                    print("❌ Failed to reset backend!")

                # Add self to party
                party = [user_info["Username"]]

                # Invite and join party members
                player_ids = []
                for member in party:
                    try:
                        # get party
                        party_res = requests.get(f"{SERVER}/get_party", params={"username": user_info["Username"]})
                        party = party_res.json().get("party", [])
                        if user_info["Username"] not in party:
                            party.append(user_info["Username"])

                        # reset party & game
                        requests.post(f"{SERVER}/reset_party", json={"username": user_info["Username"]})
                        requests.post(f"{SERVER}/restart")

                        # join & launch
                        for member in party:
                            join_res = requests.post(f"{SERVER}/join", json={"username": member})
                            if join_res.status_code == 200:
                                player_id = join_res.json().get("player_id")
                                if isinstance(player_id, list):
                                    player_id = player_id[0]

                                print(f"✅ {member} joined as {player_id}")
                                if member == user_info["Username"]:
                                    import importlib.util
                                    game_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "game_client.py"))
                                    spec = importlib.util.spec_from_file_location("game_client", game_path)
                                    game_module = importlib.util.module_from_spec(spec)
                                    sys.modules["game_client"] = game_module
                                    spec.loader.exec_module(game_module)
                                    game_module.main(player_id)
                    except Exception as e:
                        print("🔥 Error during game start:", str(e))


        # Modal Invite Popup
            finally:
                if pending_invite_from:
                    modal = pygame.Rect(200, 200, 500, 200)
                    pygame.draw.rect(screen, (255, 255, 240), modal, border_radius=12)
                    pygame.draw.rect(screen, BLACK, modal, 2, border_radius=12)
                    draw_text(f"{pending_invite_from} invited you!", 250, 240, BLACK)
                    accept_btn = draw_button("Accept", 250, 300, 100, 40, GREEN)
                    decline_btn = draw_button("Decline", 480, 300, 100, 40, RED)

                    if pygame.mouse.get_pressed()[0]:
                        pos = pygame.mouse.get_pos()
                        if accept_btn.collidepoint(pos):
                            requests.post(f"{SERVER}/accept_invite", json={"from": pending_invite_from, "to": user_info["Username"]})
                            pending_invite_from = None
                            fetch_party()
                        elif decline_btn.collidepoint(pos):
                            requests.post(f"{SERVER}/decline_invite", json={"from": pending_invite_from, "to": user_info["Username"]})
                            pending_invite_from = None

                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        update_status(False)
                        pygame.quit(); sys.exit()
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        if input_box.collidepoint(event.pos): active = True
                        else: active = False
                        if search_btn.collidepoint(event.pos):
                            result = search_user_by_username(search_text)
                            if not result:
                                search_result = "not_found"
                            elif result["Username"] == user_info["Username"] or result["Username"] in [f["Username"] for f in friends]:
                                search_result = "already_friends"
                            else:
                                search_result = result
                            message = ""
                        if reset_btn.collidepoint(event.pos):
                            try:
                                requests.post(f"{SERVER}/reset_party", json={"username": user_info["Username"]})
                                fetch_party()
                                message = "Party reset!"
                            except:
                                message = "Failed to reset party."
                    if event.type == pygame.KEYDOWN and active:
                        if event.key == pygame.K_BACKSPACE:
                            search_text = search_text[:-1]
                        elif event.key == pygame.K_RETURN:
                            result = search_user_by_username(search_text)
                            search_result = result if result else "not_found"
                        else:
                            search_text += event.unicode

        pygame.display.update()
        clock.tick(60)



if __name__ == "__main__":
    login_screen()
    menu_screen()
